In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
from ase import Atoms, Atom
from ase.io import read, write
from ase.visualize import view
from autoadsorbate import Surface, Fragment
from autoadsorbate.Surf import attach_fragment


In [ ]:
# https://github.com/basf/autoadsorbate#
from autoadsorbate import Surface, Fragment
from ase.io import write

f = Fragment('[C+](c1ccccc1)(c1ccccc1)(c1ccccc1)')
write('tmp.xyz', f.get_conformer(0))
#is the carbo cation sterically protected?

In [ ]:
import ase
from typing import Literal

def read_atoms_object(path: str):
    """reads a atomistic structure file from the  system,
    Args:
        path: string - location on system
    returns:
        ase.Atoms object
    """
    return ase.io.read(path)

def get_sites_from_atoms(atoms: ase.Atoms, mode: Literal['slab', 'particle']):
    """
    Args:
        atoms: ase.Atoms object that can be an "slab" or 'particle'. Determines all surface sites. 
    Returns:
        pandas.DataFrame containing all site information.
    """
    return Surface(atoms, mode=mode).site_df

def get_fragment(SMILES: str):
    """
    Args:
        SMILES: string of smiles that should be placed on surface sites.
    returns:
        ase.Atoms of molecule or molecular fragment, alligned relative to the site in [0,0,0]
    """
    return Fragment(SMILES).get_conformer(0)

def get_ads_slab(slab_atoms, fragment_atoms, site_dict):
    """
    Args:
        slab_atoms: ase.Atoms, atoms of slab that should host the fragment
        fragment_atoms: ase.Atoms, molecular fragment obtained from SMILES
        site_dict: dict, information about the selected site geometry
    returns:
        ase.Atoms of molecule placed on slab
    """

    ads_slab_atoms = attach_fragment(
        atoms = slab_atoms,
        site_dict = site_dict,
        fragment = fragment_atoms,
        n_rotation = 0.,
        height=1.5
        
    )

    return ads_slab_atoms


In [ ]:
# Here is my slab: <path_to_slab>
# it should bind to a top site in the center of the slab
# how does *[O+]=C=O bind?


In [ ]:
s.site_df.loc[0].to_dict()

In [ ]:
# write('/mnt/c/Users/ef/Desktop/tmp/sites.xyz', [s.view_site(i, return_atoms = True) for i in s.site_df.index.values])

In [ ]:
s.site_df

In [ ]:
from ase.build import fcc111

out_trj = []

slab = fcc111('Cu', [3,3,2], vacuum=5)

out_trj.append(slab.copy())

s = Surface(slab)
f = Fragment('ClC(=O)[O-]')

a = attach_fragment(
    atoms = s.atoms,
    site_dict = s.site_df.loc[0].to_dict(),
    fragment = f.get_conformer(0),
    n_rotation = 0.,
    height=1.5
    
)

out_trj.append(a)

write('/mnt/c/Users/ef/Desktop/tmp/tmp.xyz', out_trj)


In [ ]:
# view(a)

In [ ]:
Fragment('ClC(=O)[O-]')

In [ ]:
from ase.build import bulk, surface
from ase.io import write
import numpy as np
import random
random.seed(48) 


# Step 1: Start with a binary oxide (e.g., NiO in rocksalt structure)
nio = bulk('NiO', 'rocksalt', a=4.17)

# Step 2: Make a surface slab (e.g., (100) surface, 3 layers)
slab = surface(nio, (1, 0, 0), layers=4)
slab.center(vacuum=10, axis=2)  # add vacuum along z
slab *= [4,4,1]

# Step 3: Replace some Ni with Fe to create mixed-metal oxide slab
symbols = slab.get_chemical_symbols()
for i, sym in enumerate(symbols):
    if sym == "Ni" and np.random.rand() < 0.3:  # randomly swap ~30% Ni for Fe
        symbols[i] = "Fe"


slab.set_chemical_symbols(symbols)

s = Surface(slab)

inds = []
for t in s.site_df.topology.values:
    inds+=t
set(inds)


del_lst = random.sample(inds, 15)

slab = slab[[atom.index for atom in slab if atom.index not in del_lst]]
slab.rattle(stdev=.2)

write("NiFeO_slab.xyz", slab)
print("Saved NiFeO_slab.xyz")

In [ ]:
pwd

In [14]:
from tools.tools import read_atoms_object, md_run_atoms, relax_atoms

atoms = read_atoms_object('./NiFeO_slab.xyz')

relax_atoms(atoms)

Using Materials Project MACE for MACECalculator with /home/ef/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/ef/venvs/llm_adsorbate/lib/python3.11/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Atoms(symbols='Fe18Ni46O54', pbc=[True, True, False], cell=[[11.794541110191613, 0.0, 0.0], [5.897270555095806, 10.214372227405853, 0.0], [0.0, 0.0, 28.42642717882259]], tags=..., calculator=MACECalculator(...))